In [2]:
# !pip3 install torch torchvision torchaudio
# !pip install ultralytics

In [3]:
!nvidia-smi

Sun May  7 11:13:14 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.79                 Driver Version: 531.79       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 T...  WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   54C    P0               15W /  N/A|      0MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")

In [5]:
model.train(data="data.yaml", epochs=200, device=0)

Ultralytics YOLOv8.0.93  Python-3.10.7 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=200, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False,

Test the model

In [3]:
from ultralytics import YOLO
model = YOLO("best.pt")

In [4]:
results = model.predict("pot.jpg")


WARNING  NMS time limit 0.550s exceeded
image 1/1 D:\Programs\Projects\potholes\yolov8\pot.jpg: 640x640 3 2s, 19.1ms
Speed: 7.9ms preprocess, 19.1ms inference, 1275.5ms postprocess per image at shape (1, 3, 640, 640)


In [5]:
result = results[0]
output = []
for box in result.boxes:
    x1, y1, x2, y2 = [
        round(x) for x in box.xyxy[0].tolist()
    ]
    class_id = box.cls[0].item()
    prob = round(box.conf[0].item(), 2)
    output.append([
        x1, y1, x2, y2, result.names[class_id], prob
    ])
output

[[495, 282, 546, 314, '2', 0.95],
 [155, 352, 276, 471, '2', 0.94],
 [351, 307, 418, 337, '2', 0.85]]

In [6]:
from PIL import Image, ImageDraw, ImageFont

# Load the image
image = Image.open("pot.jpg")

# Create a drawing object
draw = ImageDraw.Draw(image)

# Define font for label text
font = ImageFont.truetype("arial.ttf", 12)

# Loop through the output array and draw bounding boxes and labels
for box in output:
    x1, y1, x2, y2, label, score = box
    
    # Draw the bounding box
    draw.rectangle([(x1, y1), (x2, y2)], outline="red", width=2)
    
    # Draw the label text
    label_text = f"{label} {score:.2f}"
    text_width, text_height = font.getsize(label_text)
    draw.rectangle(
        [(x1, y1 - text_height), (x1 + text_width, y1)],
        fill="red"
    )
    draw.text((x1, y1 - text_height), label_text, fill="white", font=font)

# Save the labeled image
image.save("output.jpg")


C:\Users\lnavn\AppData\Local\Temp\ipykernel_2060\2726096988.py:21: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  text_width, text_height = font.getsize(label_text)
